In [1]:
%run 00_display_tables.ipynb

Successfully connected to py_brew database.
+-------------+-------------------+---------------+----------------+
| customer_id |   customer_name   | customer_city | customer_state |
+-------------+-------------------+---------------+----------------+
|     100     | Jennifer Gonzalez |   Smithland   |    Arkansas    |
+-------------+-------------------+---------------+----------------+
+---------+---------------------+------------+-------------+-------+------+
| date_id |        tmstp        |    date    | day_of_week | month | year |
+---------+---------------------+------------+-------------+-------+------+
|    1    | 2020-07-04 21:23:10 | 2020-07-04 |      4      |   7   | 2020 |
+---------+---------------------+------------+-------------+-------+------+
+------------------+------------------+---------------------+--------------+---------------+
|    product_id    | product_category | product_subcategory | product_name | product_price |
+------------------+------------------+------

#### Determine the products that have experienced a significant increase in sales amount compared to the previous month. Consider a significant increase to be a growth rate of at least ***200%***. Include the product information and the corresponding months.

##### ***Hint: Combine window functions, filtering conditions, and calculations to identify products with a significant increase in sales amount compared to the previous month. Apply the growth rate threshold.***

In [6]:
%%sql

SELECT
    current_month.product_id,
    current_month.sale_month AS current_month,
    previous_month.sale_month AS previous_month,
    current_month.sales_amount AS current_sales,
    previous_month.sales_amount AS previous_sales,
    (current_month.sales_amount - previous_month.sales_amount) / previous_month.sales_amount AS growth_rate
FROM
    (
        SELECT
            s.product_id,
            MONTHNAME(d.date) AS sale_month,
            SUM(s.sales_amount) AS sales_amount
        FROM
            fact_sales s
            JOIN dim_date d ON s.date_id = d.date_id
        WHERE
            d.date >= DATE_SUB(CURDATE(), INTERVAL 2 MONTH)
        GROUP BY
            s.product_id,
            MONTH(d.date),
            d.date
    ) AS current_month
    JOIN
    (
        SELECT
            s.product_id,
            MONTHNAME(d.date) AS sale_month,
            SUM(s.sales_amount) AS sales_amount
        FROM
            fact_sales s
            JOIN dim_date d ON s.date_id = d.date_id
        WHERE
            d.date >= DATE_SUB(CURDATE(), INTERVAL 3 MONTH)
            AND d.date < DATE_SUB(CURDATE(), INTERVAL 1 MONTH)
        GROUP BY
            s.product_id,
            MONTH(d.date),
            d.date
    ) AS previous_month ON current_month.product_id = previous_month.product_id
WHERE
    -- growth_rate >= 0.5
    (current_month.sales_amount - previous_month.sales_amount) / previous_month.sales_amount >= 2;



product_id,current_month,previous_month,current_sales,previous_sales,growth_rate
COL:ICE:ICE:1076,March,February,47.81,13.89,2.442045
HOT:TEA:BLA:1196,April,March,48.07,11.54,3.165511
HOT:TEA:CHA:1537,April,March,16.65,5.00,2.330000
RET:KIT:FRE:1737,April,April,47.60,7.40,5.432432
SNA:BAK:COO:1525,April,April,31.86,6.52,3.886503
SNA:BAK:CIN:1787,April,March,30.69,6.00,4.115000
RET:KIT:FRE:1202,April,March,48.81,15.70,2.108917
HOT:TEA:BLA:1953,April,February,34.67,9.13,2.797371
HOT:TEA:BLA:1953,March,February,40.76,9.13,3.464403
COL:ICE:ICE:1641,May,February,37.48,7.67,3.886571
